# Motif Discovery with EM
(50 pts) An example of the EM algorithm for computational biology is discussed in __Section 2__ of the paper [The EM Algorithm and the Rise of Computational Biology](http://www.people.fas.harvard.edu/~junliu/TechRept/10folder/Fan&Yuan&Liu10.pdf). The algorithm is implemented below.
<!-- Code for setting up the problem, simulating sequences (data), and comparing results is given. The notation is somewhat cumbersome, as is usually the case with EM. Also, not all steps are explicit in the paper, but for the most part the derivation is given.  -->

__Note:__ The variable $Z$ in the code below is related to $\Gamma$ in the paper.

Your task is to read the relevant section in the paper, answer the following questions, and understand and execute the code given below.

1. What is the (observed) data for this problem? (Transcription factor binding sites, i.e. TFBSs, $Y_k$)
2. What is the hidden data? (Motif location, $Γ_{k,l}$)
3. What is the distribution of the hidden data? Clearly indicate the parameters of the distribution, if any? ($Γ_{k,l} \sim \text{Bernuli}(\frac{1}{L_k'})$, where $L_k' = L_k - w + 1$, $L_k'$ are possible positions that a motif site of length $w$ may start, $L_k$ is the total length of the sequence, $w$ is the length of the motif site.)
4. What is the conditional distribution of the observed data given the hidden data? What are the parameters of this distribution if any? (Multinomial distribution, $𝐘_k|Γ_{k,l}=1,𝚯\sim Multinomial(𝐡(𝐁_{k,l}),\ldots,𝐡(𝐘 _{k,l+j+1});𝛉_0,𝛉_1,\ldots,𝛉_i)$)
5. What does the quantity $\frac{w_{k,l}}{W_k}$ in line 11 of page 5 represent?
(The probability of $Y_{k,l}$ as the start point)
6. Identify the lines in the code below concerned with computing $w_{k,l}$ and $W_k$.
7. Using the provided code, perform the following tasks at least two times:  
   - Generate a set of parameters for this problem at random.
   - Produce synthetic data (both observed and hidden data).
   - Find the maximum-likelihood estimate for the parameters based on complete data.
   - Find the EM estimate for the parameters based on observed data.
   - Find the MSE for both ML and EM estimates and compare. 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats as st
from collections import Counter
print("Modules Imported!")

Modules Imported!


In [3]:
# Setting up the constants
w = 6 # length of motif
N = 20 # number of sequences
len_lu = [w,w+10] # lower and upper bound on the lengths of sequences
m = 4 # alphabet size


In [4]:
# generating theta
def gen_theta(w,m):
    theta = [[0]*m]*(w+1)
    for i in range(w+1):
        th = np.concatenate(([0],st.uniform.rvs(0,1,m-1),[1]))
        th = np.sort(th)
        theta[i] = np.diff(th)
    return theta

In [5]:
# generating RVs and sequences

# generating one sequence
def genSeq(n,p): 
    # Generate an iid random sequences of length n over 
    # the alphabet 0,1,...,|p|-1 where the probability of i is p[i]
    y = [0] * n
    for i in range(n):
        r = st.uniform.rvs()
        y[i] = np.sum(r>np.cumsum(p))
    return y

# generate N sequences containing motifs at random positions
def genSeqMotif(theta,N,len_lu):
    w = len(theta) - 1
    Y = [[]]*N # sequences
    Z = [0]*N # start positions of motifs
    L = [0]*N # lengths of sequences
    for i in range(N):
        L[i] = st.randint.rvs(len_lu[0],len_lu[1]+1)
        Z[i] = st.randint.rvs(0,L[i]-w+1)
        y_nonmotif_pre = genSeq(Z[i],theta[0])
        y_motif = [genSeq(1,theta[i+1])[0] for i in range(w)]     
        y_nonmotif_post = genSeq(L[i]-Z[i]-w, theta[0])
        Y[i] = np.concatenate((y_nonmotif_pre,y_motif,y_nonmotif_post)).astype(int)
    return (Y,Z)

In [6]:
# estimators

# Below is the estimator with complete data. You can use some of this code for the incomplete data. However while here we 
# know the value of z as z=Z[i], for EM we need to do the computation for all possible z and take expectation. Understanding 
# completeDataML is a good first step for writing the EM code
def completeDataML(Y,Z,w,m):
    thetaHatML = [[0]*m] * (w+1)
    hn = np.array([[0]*m] * (w+1)) # composition for one sequence
    h = np.array([[0]*m] * (w+1)) # compostion for all sequences
    N = len(Y)
    for n in range(N):
        z = Z[n]
        y = Y[n]
        y_nonmotif = np.concatenate((y[0:z],y[z+w:])).astype(int)
        y_motif = y[z:z+w]
        hn[0] = [Counter(y_nonmotif)[j] for j in range(m)]
        for i in range(1,w+1):
            hn[i] = [Counter([y_motif[i-1]])[j] for j in range(m)]
        h += hn          
    thetaML = [h[i]/sum(h[i]) for i in range(w+1)]  
    return thetaML
    
    
# This module is used in motifDiscoveryEM below to
# find the expected counts given the sequence y and the 
# current estimate of the parameteres
def expectedCounts(y,theta_current):
    w = len(theta_current) - 1
    m = len(theta_current[0])
    expCnts = np.array([[0.0]*m]*(w+1))
    hz = np.array([[0]*m]*(w+1))  # the composition for sequences y for a 
    # given value of z; hz = [hz[0],hz[1],...,hz[w]]
    Prz = np.array([0.0]*(len(y)-w+1)) # probability for a given value of z
    for z in range(len(y)-w+1):
        y_nonmotif = np.concatenate((y[0:z],y[z+w:])).astype(int)
        y_motif = y[z:z+w]
        hz[0] = [Counter(y_nonmotif)[j] for j in range(m)]
        for i in range(1,w+1):
            hz[i] = [Counter([y_motif[i-1]])[j] for j in range(m)]
        Prz[z] = np.prod([np.prod( np.power(theta_current[i],hz[i])) for i in range(w+1)]) # w_{k,l}
        expCnts += hz.astype(float)*Prz[z] # W_k = sum(Prz)
    expCnts = expCnts/sum(Prz) 
    return expCnts

# EM motif discovery, stating with theta_start and doing itr_max iterations
def motifDiscoveryEM(Y,theta_start,itr_max):
    theta_current = theta_start
    w = len(theta_current) - 1
    m = len(theta_current[0])
    for itr in range(10):
        h = np.array([[0.0]*m]*(w+1))
        for n in range(N):
            hn = expectedCounts(Y[n],theta_current)
            h += hn
        theta_current = [h[i]/sum(h[i]) for i in range(w+1)]
    return(theta_current)

In [7]:
theta_start = [[1/m]*m]*(w+1)
np.set_printoptions(precision=2,suppress=True) # printing prettier matrices

# Begin Solution
def Cal_MSE(t_hat,t_real, col, row):
  diff = np.array(t_hat) - np.array(t_real)
  summ = 0
  for i in range(0,col):
    for j in range(0,row):
      summ = summ + (diff[i][j] ** 2)
  MSE = summ / (row*col)
  return MSE

MSE_ML_lis = []
MSE_EM_lis = []
for t in range(20): # run 20 times
  Theta = gen_theta(w,m)
  (YY,ZZ) = genSeqMotif(Theta,N,len_lu)
  Theat_hat = completeDataML(YY,ZZ,w,m)
  (Theta_current) = motifDiscoveryEM(YY,theta_start,100)
  MSE_ML = Cal_MSE(Theat_hat, Theta, w+1, m)
  MSE_EM = Cal_MSE(Theta_current, Theta, w+1, m)
  MSE_ML_lis.append(MSE_ML)
  MSE_EM_lis.append(MSE_EM)
  print('The real value of theta in run %d:\n%s' % (t+1, np.array(Theta)))
  print('The ML of theta in run %d:\n%s' % (t+1, np.array(Theat_hat)))
  print('The ME of theta in run %d:\n%s' % (t+1, np.array(Theta_current)))
  print('\n')

print('All MSE_MLs of theta:\n%s' % MSE_ML_lis) # put MSE_ML at each cycle together
print('All MSE_EMs of theta:\n%s' % MSE_EM_lis) # put MSE_EM at each cycle together
# End Solution


The real value of theta in run 1:
[[0.25 0.52 0.08 0.15]
 [0.13 0.   0.74 0.13]
 [0.03 0.17 0.13 0.67]
 [0.03 0.05 0.31 0.61]
 [0.1  0.4  0.41 0.09]
 [0.6  0.16 0.2  0.04]
 [0.26 0.53 0.03 0.18]]
The ML of theta in run 1:
[[0.31 0.45 0.08 0.16]
 [0.2  0.   0.75 0.05]
 [0.   0.05 0.   0.95]
 [0.   0.05 0.5  0.45]
 [0.1  0.25 0.55 0.1 ]
 [0.7  0.05 0.2  0.05]
 [0.3  0.35 0.1  0.25]]
The ME of theta in run 1:
[[0.3  0.46 0.09 0.16]
 [0.21 0.   0.72 0.07]
 [0.   0.   0.   1.  ]
 [0.05 0.04 0.47 0.44]
 [0.09 0.3  0.55 0.06]
 [0.65 0.05 0.24 0.06]
 [0.39 0.3  0.07 0.24]]


The real value of theta in run 2:
[[0.05 0.18 0.24 0.53]
 [0.21 0.05 0.14 0.6 ]
 [0.1  0.25 0.39 0.27]
 [0.42 0.44 0.1  0.05]
 [0.25 0.08 0.5  0.17]
 [0.12 0.2  0.15 0.52]
 [0.3  0.31 0.24 0.15]]
The ML of theta in run 2:
[[0.03 0.19 0.28 0.5 ]
 [0.3  0.   0.   0.7 ]
 [0.1  0.3  0.25 0.35]
 [0.45 0.45 0.1  0.  ]
 [0.35 0.05 0.4  0.2 ]
 [0.1  0.2  0.35 0.35]
 [0.2  0.3  0.25 0.25]]
The ME of theta in run 2:
[[0.03 0.13 0.28